In [21]:
import math
import random

# Ex 2.1

### Creating the DGIM class

In [32]:
class DGIM():
    def __init__(self, N):
        self.N = N
        # the index is the power of the 2 corresponding to the number of ones in the buckets
        # Ex: 0: 2^0 = 1 -> the buckets only have one 1 each.
        self.bucket_list = {0: []}
        # current max power of 2, indicates the biggest current bucket size
        self.max_power = 0
        # helper flag used to discard elements that are N bits old
        self.flag = 0
        self.timestamp = 0
        
        
    def process_bit(self, bit):
        
        if self.timestamp > 0: # to ensure that we don't access a empty list
            if self.bucket_too_old():
                # remove the oldest bit
                self.bucket_list[self.max_power] = self.bucket_list[self.max_power][:-1]
                
                # if the largest (oldest) bucket becomes empty we remove it
                if len(self.bucket_list[self.max_power]) == 0:
                    self.bucket_list.pop(self.max_power)
                    self.max_power -= 1
                    
        if bit == 0:
            return
        
        self.update()
        self.timestamp += 1
        self.flag = self.timestamp % self.N
        
        return
    
    
    def update(self):
        power = 0
        
        # add to the begining of the list (queue)
        self.bucket_list[0] = [[self.timestamp, self.flag]] + self.bucket_list[0]
        
        # when we have more than 2 buckets of the same size we 
        # need to combine them into greater power buckets
        # using while instead of for prevents us from iterating through all of the buckets
        while len(self.bucket_list[power]) > 2:
            # get the buckets to be merged
            bucket = self.bucket_list[power][-2:]
            # the newest element remains in the bucket
            self.bucket_list[power] = self.bucket_list[power][:1]
            power += 1
            
            if power in self.bucket_list:
                self.bucket_list[power] = [bucket[0]] + self.bucket_list[power]
            else:
                self.bucket_list[power] = [bucket[0]]
                self.max_power = power
                
        return
        
    
    def get_count(self):
        result = 0
        power = 1
        
        for x in range(self.max_power + 1):
            # number of buckets of size x, varies between 0 and 2
            bucket_len = len(self.bucket_list[x])
            if bucket_len > 0:
                result += bucket_len * power
            # increase the power of 2 (ex: 2^1 becomes 2^2)
            power *= 2
        # since we only consider half of the last bucket 
        # we need to subtract half of it from the result
        result -= math.floor(self.max_power/2)
            
        return result
    
    def get_in_last_k(self, k):
        timepoint = self.timestamp - k
        result = 0
        power = 1
        # auxiliary parameters to remove half of B
        B_timestamp = math.inf
        B_power = 0
        
        for k in range(self.max_power + 1):
            for v in range(len(self.bucket_list[k])):
                if self.bucket_list[k][v][0] > timepoint:
                    result += power
                    # keep track of the earliest bucket that overlaps with k
                    if self.bucket_list[k][v][0] < B_timestamp:
                        B_timestamp = self.bucket_list[k][v][0]
                        B_power = power
                        
            power *= 2
            
        # since we already summed all of B now we need to remove half of it
        result -= math.floor(B_power/2)      
        
        return result
    
    def bucket_too_old(self):
        # returns True or False
        # bucket_list[self.max_power][-1] -> last element of the oldest bucket, that is, the oldest element
        # this element is only equal to the value of the flag when N elements have gone through the algorithm
        return (self.flag == self.bucket_list[self.max_power][-1][1])
    
    
    

### Creating a function to simulate a stream

In [114]:
def test_DGIM(N=20000, k=2000, interval=1000):
    dgim = DGIM(N=N)
    counter = 0
    ones = 0
    end = 0
    
    while True:
        if end > N:
            break
        
        x = random.randint(0, 1)
        if x == 1:
            ones += 1
        dgim.process_bit(x)
        
        if counter == interval:
            print(f'###################\nReal count: {ones}')
            print(f'DGIM estimate: {dgim.get_count()}')
            print(f'In the last {k} elements: {dgim.get_in_last_k(k)}')
            counter = 0    
            
        counter += 1
        end += 1

### Testing the algorithm

In [116]:
test_DGIM(10000, 200, 200)

###################
Real count: 93
DGIM estimate: 91
In the last 200 elements: 77
###################
Real count: 198
DGIM estimate: 195
In the last 200 elements: 166
###################
Real count: 282
DGIM estimate: 279
In the last 200 elements: 218
###################
Real count: 381
DGIM estimate: 378
In the last 200 elements: 221
###################
Real count: 486
DGIM estimate: 483
In the last 200 elements: 198
###################
Real count: 584
DGIM estimate: 580
In the last 200 elements: 168
###################
Real count: 667
DGIM estimate: 663
In the last 200 elements: 219
###################
Real count: 783
DGIM estimate: 779
In the last 200 elements: 207
###################
Real count: 886
DGIM estimate: 882
In the last 200 elements: 214
###################
Real count: 981
DGIM estimate: 977
In the last 200 elements: 181
###################
Real count: 1073
DGIM estimate: 1069
In the last 200 elements: 241
###################
Real count: 1172
DGIM estimate: 1168
In the la